# Basics of NFA

You may wish to watch my lecture on NFA a little, then play with the
code below, and then finish watching.

In [ ]:
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
import sys

# -- Detect if in Own Install or in Colab
try:
    import google.colab
    OWN_INSTALL = False
except:
    OWN_INSTALL = True
    
if OWN_INSTALL:
    
  #---- Leave these definitions ON if running on laptop
  #---- Else turn OFF by putting them between ''' ... '''

  sys.path[0:0] = ['../../../../..',  '../../../../../3rdparty',  
                   '../../../..',  '../../../../3rdparty',  
                   '../../..',     '../../../3rdparty', 
                   '../..',        '../../3rdparty',
                   '..',           '../3rdparty' ]

else: # In colab
  ! if [ ! -d Jove ]; then git clone https://github.com/ganeshutah/Jove Jove; fi
  sys.path.append('./Jove')
  sys.path.append('./Jove/jove')

# -- common imports --
from jove.DotBashers import *
from jove.Def_md2mc  import *
from jove.Def_NFA    import *
from jove.Def_DFA    import *
from jove.AnimateNFA import *
from jove.Def_RE2NFA import *
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

# Why NFA?

## 1) NFA emphasize the essence of the language of interest
## 2) By exploiting the ability to be in a set of states, it remains compact

## A good example is "third last is a 1" i.e.
### {0,1}* 1 {0,1} {0,1}

## A DFA for this language ends up keeping the full history of 3 past bits
### and hence prove to be exponentially large (2^3 states)

### For 4th-last is a 1, the NFA will have 16 states

### This is one example of when a DFA becomes large
### More importantly, this DFA is very hard to trust ... its state semantics cannot be reliably argued
### And one is better off designing a DFA with more states (emphasizing clarity) and then minimizing

In [ ]:
ThirdLastIs1DFA = md2mc(
'''
DFA

I      :  0   -> I    !! A 0 need not be remembered
I      :  1   -> S1   !! A 1 needs to be remembered

S1     :  0   -> S10  !! Record where the 1 is
S1     :  1   -> S11  !! ditto

S10    :  0   -> F00  !! The 1 has been pushed to the "third last"
S10    :  1   -> F01  !! ditto

S11    :  0   -> F10  !! The 1 has been pushed to the third last
S11    :  1   -> F11  !! ditto

F00    :  0   -> I   !! The 1 is gone from the picture (no longer in the third-last)
F00    :  1   -> S1  !! ditto

F01    :  0   -> S10  !! The 1 is gone from the picture 
F01    :  1   -> S11  !! ditto

F10    :  0   -> F00  !! There is a 1 in the third-last position now
F10    :  1   -> F01  !! ditto

F11    :  0   -> F10  !! There is a 1 in the third-last position
F11    :  1   -> F11  !! ditto

'''
)

In [ ]:
dotObj_dfa(ThirdLastIs1DFA, FuseEdges=True)

## An NFA avoids the specification complexity, but requires the use of nondeterminism
### A big plus is that it is VERY easy to trust

In [ ]:
ThirdLastIs1NFA = md2mc('''
NFA  

I   : 0|1 -> I

I   : ''  -> S

S   : 1   -> Sa

Sa  : 0|1 -> Sb

Sb  : 0|1 -> F

''')

# Study the third-last-is-1

Put it in animate mode, and then single-step to see the EClosure of states

In [ ]:
AnimateNFA(ThirdLastIs1NFA, FuseEdges=True)
display(HTML('<link rel=\"stylesheet\" href=\"//stackpath.bootstrapcdn.com/font-awesome/4.7.0/css/font-awesome.min.css\"/>'))

# An alternate NFA for "third last is 1"

In [ ]:
ThirdLastIs1NFAalt = md2mc('''
NFA  

I   : 0|1 -> I

I   : 1   -> Sa

Sa  : 0|1 -> Sb

Sb  : 0|1 -> F

''')

In [ ]:
dotObj_nfa(ThirdLastIs1NFAalt, FuseEdges=True)

In [ ]:
help(nfa2dfa)

In [ ]:
nfa2dfa(ThirdLastIs1)

In [ ]:
dotObj_dfa(nfa2dfa(ThirdLastIs1), FuseEdges=True)

In [ ]:
nfa2dfa(ThirdLastIs1, STATENAME_MAXSIZE=30)

In [ ]:
help(dotObj_dfa)

In [ ]:
dotObj_dfa(nfa2dfa(ThirdLastIs1, STATENAME_MAXSIZE=30), STATENAME_MAXSIZE=30)

In [ ]:
dotObj_dfa(nfa2dfa(ThirdLastIs1, STATENAME_MAXSIZE=30), STATENAME_MAXSIZE=30).render('ThirdLastDFA.pdf')

In [ ]:
DFA_a_bc_ca = md2mc(
'''
DFA

I : a -> F
I : b -> Sb
I : c -> Sc

Sb : c -> F
Sb : a|b -> BH

Sc : a   -> F
Sc : b|c -> BH

F  : a|b|c -> BH

BH : a|b|c -> BH
''')

In [ ]:
dotObj_dfa_w_bh(DFA_a_bc_ca, FuseEdges=True)

In [ ]:
NFA_a_bc_ca = md2mc(
'''
NFA

I1 : a -> F1

I2 : b -> Sb
Sb : c -> F2

I3 : c -> Sc
Sc : a -> F3
''')

In [ ]:
dotObj_nfa(NFA_a_bc_ca)

In [ ]:
AnimateNFA(NFA_a_bc_ca, FuseEdges=True)
display(HTML('<link rel=\"stylesheet\" href=\"//stackpath.bootstrapcdn.com/font-awesome/4.7.0/css/font-awesome.min.css\"/>'))

In [ ]:
dotObj_nfa(re2nfa("(a+bc+ca)(a+bc+ca)")).render("a_bc_ca_concat.pdf")

In [ ]:
dotObj_dfa(nfa2dfa(NFA_a_bc_ca))

# This is an example from Figure 7.7 of the book

I am choosing to type it in two different ways: one with a single initial state, and 
another with multiple initial states. Fig77 is the single initial state version, and
Fig77a is with multiple initial states

In [ ]:
Fig77 = md2mc('''NFA
I : '' -> A, G
A : '' -> B, C
G : 1  -> F
B : 1  -> D
C : 0  -> E
D : '' -> A, G
E : '' -> A, G
''')

In [ ]:
AnimateNFA(Fig77, FuseEdges=False)
display(HTML('<link rel=\"stylesheet\" href=\"//stackpath.bootstrapcdn.com/font-awesome/4.7.0/css/font-awesome.min.css\"/>'))

In [ ]:
Fig77a = md2mc('''NFA
IA : '' -> B, C
IG : 1  -> F
B : 1  -> D
C : 0  -> E
D : '' -> IA, IG
E : '' -> IA, IG
''')

In [ ]:
dotObj_nfa(Fig77a)

In [ ]:
dotObj_dfa(nfa2dfa(Fig77a, STATENAME_MAXSIZE=50), STATENAME_MAXSIZE=50)

In [ ]:
# Our automd markdown works for NFA also. Here is the NFA for
# the language "ends in 0101"

NFA_ends0101 = md2mc(''' 
!!---------------------------------------------------------------------------
!! This NFA looks for patterns of the form 
!!
!!     ......<some 0's and 1's>.....0101
!!
!! i.e., all strings that end in 0101.
!!
!! This example (over and above thirdlastis1.nfa) also illustrates
!! the power of nondeterminism. Anytime we see a 0 from state I, we
!! can have the NFA speculate that it is seeing the beginning of a
!! 0101 pattern! If the NFA fails in its guess, it pays no price
!! (because another guess is soon coming from behind).
!!
!! * We take 4 states in the NFA solution. The DFA solution takes
!!   5 states.
!!
!! * This shows how burden is shifted from the
!!   human to the computer doing the NFA simulation.
!!   Even though the savings in states is not dramatic,
!!   the effort to build this NFA is much lower than building
!!   the corresponding DFA.
!! 
!!-----------------------------------------------------------------
!! For NFA, each line can be as follows:
!!
!! State : in1 | in2 ...  ->  COmma, Separated, State, List  !! comment
!!-----------------------------------------------------------------

NFA

I    : 1   -> I      !! I goes to I upon 1

I    : 0   -> S0, I  !! I goes to S0 and I upon 0

S0   : 1   -> S01    !! S0 goes to S01 upon 1

S01  : 0   -> S010   !! S01 goes to S010 upon 0

S010 : 1   -> F      !! S010 goes to F upon F

!!---
''')

In [ ]:
# Now let us see the dict version of the NFA
NFA_ends0101 

In [ ]:
# Let's see the drawing of this NFA

dotObj_nfa(NFA_ends0101)  

In [ ]:
# The drawing produces separated self-loops. Let's fuse 'em

# Now we are going to fuse the separate self-loops into one

dotObj_nfa(NFA_ends0101, FuseEdges=True) 

# NFA for 3rd-last is 1

In [ ]:
# Here is the automd (markdown) for an NF whose third-last symbol is a 1

NFA_3rdlast_1 = md2mc('''
NFA

I : 0  | 1  -> I  !! Upon a 0 or a 1, continue to stay in state I

I : ''      -> S0 !! Nondeterministically transition to state S0

S0 : 1      -> S1 !! This is the key "1" (third-last being 1) seen

S1 : 0 | 1  -> S2 !! One 0/1 takes the NFA to S2

S2 : 0 | 1  -> F  !! One more 0/1 takes the NFA to F

''')

In [ ]:
# Let us see its internal representation

NFA_3rdlast_1

In [ ]:
# Let us see this NFA drawn out

dotObj_nfa(NFA_3rdlast_1, FuseEdges=True)

In [ ]:
# Let's step the NFA on epsilon   
step_nfa(NFA_3rdlast_1, "I", "")

In [ ]:
# Let's step it on '0'  
step_nfa(NFA_3rdlast_1, "I", "0")

In [ ]:
# Step it on a "1" to see two next states. That shows nondeterminism!

step_nfa(NFA_3rdlast_1, "I", "1")  

In [ ]:
# What is the Eclosure of the set of states {I} ?

Eclosure(NFA_3rdlast_1, {"I"})

In [ ]:
help(accepts_nfa)

In [ ]:
accepts_nfa(NFA_3rdlast_1, "0100100", chatty=True)

# Let us run NFA interactively via widgets

We have two NFA to play with, namely nfa_ends0101 and NFA_3rdlast_1


In [ ]:
import ipywidgets as wdg
def run_nfa_slider(N, s, n):
    """Run NFA N from N["Q0"] (which is a set..) on substring s[0:n]
    """
    S = N["Q0"] 
    if (n > len(s)):
        n = len(s)
    print("string = ", s[0:n])
    run_nfa(N, S, s[0:n], True)
    
def run_nfa_int(N1, N2):
    """Run interactively from the given NFA .. from {q0} 
       on input string's substring
       as picked by slider.
    """
    inp = input("Please provide string: ")
    wdg.interact(run_nfa_slider, N = {'N1': N1, 'N2': N2}, 
                 s = inp, n=(0,32))
    
def test_run_nfa_int(N1, N2):
    """Test the interactive run of NFA.
    """
    run_nfa_int(N1, N2)

#-- enable this later. test_run_nfa_int()

## Once you run the command below, provide a string

Provide a string of 0's and 1's. Then you can use the pull-down to pick the NFA
to test, and the slider to incrementally consume more of the input string


In [ ]:
test_run_nfa_int(NFA_ends0101, NFA_3rdlast_1)

## Alternate definition of the NFA of Figure 7.7 is below
## Illustrates the use of multiple initial states

 

In [ ]:
Fig77alt = md2mc(

'''NFA
!! --- GONE !!   ---> I : '' -> A, G

!! --- replace  A : '' -> B, C  by
IA : '' -> B, C

!! --- replace G : 1  -> F  by
IG : 1  -> F

B : 1  -> D
C : 0  -> E

D : '' -> IA, IG  !! suitably changed

E : '' -> IA, IG  !! suitably changed
''')

In [ ]:
dotObj_nfa(Fig77alt, FuseEdges="True")